In [6]:
#Click cntr + F9 to run, (ignore warning)max_payoff=82 #maximum odd, mostly for old accounts with max bets
bet_amt = 12772
Round = 7722

In [10]:
#@title This is the code, hidden from sight for neatness
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import numpy as np
from datetime import date
import os,io
var = 'close'
alpha= 0.05
arena_threshold=1.03 #minimum expectation to consider the bet
max_payoff=82 #maximum odd, mostly for old accounts with max bets
var_used = 'Expected_profit_at_'+var
import re
params = ",0\nopen_odds_norm,6.138644539364521\nname_Admiral Blackbeard,-2.777364345485839\nname_Bonnie Pip Culliford,-2.792773814976431\nname_Buck Cutlass,-2.7711556395900465\nname_Captain Crossblades,-2.993594602702175\nname_Fairfax the Deckhand,-2.9067730071515285\nname_Federismo Corvallio,-2.8091537880541044\nname_Franchisco Corvallio,-2.87126226562561\nname_Gooblah the Grarrl,-2.685502707943747\nname_Lucky McKyriggan,-2.889732930890241\nname_Ned the Skipper,-2.7570793336587656\nname_Ol' Stripey,-2.8580812159616387\nname_Orvinn the First Mate,-2.9795749094534196\nname_Peg Leg Percival,-2.905826990593217\nname_Puffo the Waister,-2.926613367682632\nname_Scurvy Dan the Blade,-2.837227504608177\nname_Sir Edmund Ogletree,-2.847380322589961\nname_Squire Venable,-3.415756122210414\nname_Stuff-A-Roo,-3.331547349136995\nname_The Tailhook Kid,-2.8161407329422348\nname_Young Sproggie,-2.818935711229024\n"


params = pd.read_csv(io.StringIO(params))

params.rename(columns={params.keys()[0]:'name',params.keys()[1]:'value'},inplace=True)
pd.set_option('display.max_columns', 20)

def cleanhtml(raw_html):
  cleanr = re.compile('<.*?>')
  cleantext = re.sub(cleanr, '\t', raw_html)
  return cleantext

name = Round


dictt = {6:'Peg Leg Percival',
         2:'Young Sproggie',
         9:'Stuff-A-Roo',
         13:'Ned the Skipper',
         8:'Puffo the Waister',
         14:'Fairfax the Deckhand',
         16:'Franchisco Corvallio',
         12:"Ol' Stripey",
         11:'Captain Crossblades',
         3:'Orvinn the First Mate',
         5:'Sir Edmund Ogletree',
         4:'Lucky McKyriggan',
         18:'Admiral Blackbeard',
         10:'Squire Venable',
         1:'Scurvy Dan the Blade',
         7:'Bonnie Pip Culliford',
         19:'Buck Cutlass',
         15:'Gooblah the Grarrl',
         20:'The Tailhook Kid',
         17:'Federismo Corvallio'}

import os,json

if True:
  try:
    with open('./'+str(name)+'.json') as f:
        data = json.load(f)
    if 0 in data['winners']:
       os.system('curl -o ./%s.json https://foodclub.neocities.org/rounds/%s.json '%(name,name))
       
  except:
    os.system('curl -o ./%s.json https://foodclub.neocities.org/rounds/%s.json '%(name,name))


def f(name=name):
    with open('./'+str(name)+'.json') as f:
      data = json.load(f)
      #print (data)
    #0,game,win/lose,name,odds,open_odds,close_odds,food_adjustment,sum_(1/open_odds),sum_(1/close_odds)

    df=pd.DataFrame(np.concatenate(data['pirates']),columns=['name'])
    df['game'] = name
    df['name'] =df['name'].map(dictt)
    #df[0] = str(data)
    df['open_odds'] = np.concatenate([x[1:] for x in data['openingOdds']])
    df['close_odds'] = np.concatenate([x[1:] for x in data['currentOdds']])
    def get_winner(winner):
        array = ['Loser',]*20
        for i in range(len(winner)):
            array[(i)*4+(winner[i]-1)%4] = 'Winner'
        return array
    df['win/lose'] = get_winner(data['winners'])
    for i in range(0,20,4):
        df.loc[i:i+3,'sum_(1/open_odds)'] =np.sum(1/df.loc[i:i+3,'open_odds'])
        df.loc[i:i+3,'sum_(1/close_odds)']=np.sum(1/df.loc[i:i+3,'close_odds'])
    df['open_odds_norm'] = (1/df['open_odds'])/df['sum_(1/open_odds)']
    df['close_odds_norm'] = (1/df['close_odds'])/df['sum_(1/close_odds)']


    df['coefficient'] = df['name'].apply(lambda x :\
                       [params[params['name']==y]['value'].values[0]
                        for y in params['name'] if x in y])
    df['logit'] = df['open_odds_norm']*params[params['name']=='open_odds_norm']['value'].values[0]+\
                    df['coefficient'].apply(lambda x : x[0])
    del df['coefficient']
    df['prob'] = 1/(1+np.exp(-df['logit']))


    df['prob_win'] = df['prob']

    for i in range(0,len(df),4):
      df.loc[i:i+3,'pred'] = df['prob_win']/np.sum(df.loc[i:i+3,'prob_win'])
    df=df.copy()
    df['Expected_profit_at_close'] = df['pred']*df['close_odds']
    df['Expected_profit_at_open'] = df['pred']*df['open_odds']
    if 0 not in list(data['winners']):
        df['win'] = 0
        for i in range(len(data['winners'])):
            df.loc[i*4+data['winners'][i]-1,'win'] = 1   
    else:
      df['win'] = np.nan
    return df
df= f(name)

df_all_combs=pd.DataFrame([0,1,2,3,4],columns=[0,])
for i in range(1,5):
    temp = df_all_combs.copy()
    for val in range(5):
        temp[i] = val
        df_all_combs = df_all_combs.append(temp)
    df_all_combs  = df_all_combs.dropna().astype(np.int32)
if True:
    i=0
    game1 = df.iloc[i:i+4]
    game2 = df.iloc[i+4:i+8]
    game3 = df.iloc[i+8:i+12]
    game4 = df.iloc[i+12:i+16]
    game5 = df.iloc[i+16:i+20]
    dictt={}
    counter =0 

    for game in [game1,game2,game3,game4,game5]:
        dictt[counter] = [np.concatenate([game['pred'].values,np.array([1,])]), #predictions
                          np.concatenate([game[var+'_odds'].values,np.array([1,])]), #closing odds (predicted)
                          np.concatenate([game['close_odds'].values,np.array([1,])]), #closing odds (actual)
                          np.concatenate([game['win'].values,np.array([1,])])] #correct choice, choosing no priates also correct
        if np.max(game[var_used]>arena_threshold)==1:
            
            play_id = game[game[var_used]==np.max(game[var_used])].index
            df.loc[play_id,'play'] = 1
        counter += 1

    temp = df_all_combs.copy()
    for j in range(5):
        temp['prob_'+str(j)] =temp[j].apply(lambda x : dictt[j][0][x])
        temp['odds_p_'+str(j)] =temp[j].apply(lambda x : dictt[j][1][x])
        temp['odds_a_'+str(j)] =temp[j].apply(lambda x : dictt[j][2][x])
        temp['win_'+str(j)] =temp[j].apply(lambda x : dictt[j][3][x])
    temp['correct']=    temp['win_0']*temp['win_1']* temp['win_2']* temp['win_3']* temp['win_4']    
    temp['payoff_p'] = temp['odds_p_0']*temp['odds_p_1']* temp['odds_p_2']* temp['odds_p_3']* temp['odds_p_4']
    temp['payoff_c'] = temp['odds_a_0']*temp['odds_a_1']* temp['odds_a_2']* temp['odds_a_3']* temp['odds_a_4']
    temp['prob'] =  temp['prob_0']*temp['prob_1']* temp['prob_2']* temp['prob_3']* temp['prob_4']
    temp['Expected'] = temp['payoff_c']*temp['prob'] #predicted earnings
    temp['Actual']= (temp['win_0']*temp['win_1']* temp['win_2']* temp['win_3']* temp['win_4'])==1
    temp = temp[(temp['payoff_c']<max_payoff)&(temp['Expected']>arena_threshold)].sort_values('Expected').reset_index(drop=1)
    threshold = temp.iloc[-min(10,len(temp))]['Expected']
    temp2 = temp[temp['Expected'] >= threshold -alpha] #you bet on these 10 if >10
    temp2['chosen'] = 0
    while np.sum(temp2['chosen']) <10 and np.mean(temp2['chosen']) < 1:
      #iteratively choose bet 
      max_prob = temp2[temp2['chosen']==0]['Expected'].max()
      candidates = temp2[(temp2['chosen']==0) & (temp2['Expected']>=max_prob-alpha)]
      max_prob = candidates['prob'].max()
      chosen = candidates[candidates['prob']==max_prob ].index
      temp2.loc[chosen,'chosen']=1
    temp2 = temp2.copy().sort_values(['chosen','Expected']).reset_index(drop=1)
    #temp2 =temp2[temp2['chosen']==1].reset_index(drop=1)


df['Arena'] = ['ShpWr',]*4+['Lgn',]*4+['TrIs',]*4+['HdnCv',]*4+['HpnHry']*4
df['O_odds'] = df['open_odds']
df['C_odds'] = df['close_odds']
df['Expected_profit'] = df['close_odds']*df['pred']-1

df3=df.copy()
df3.loc[20,['Arena','name']]=['','odds']
df3.loc[21,['Arena','name']]=['','prob/%']
df3.loc[22,['Arena','name']]=['','Expected/%']
df3.loc[23,['Arena','name']]=['','Actual']

df4=pd.DataFrame([None,]*len(temp2),columns=[1])
for i in range(1,6):
  df4[i] = ''


for i in range(1,1+len(temp2)):
  a,b,c,d,e=temp2[range(5)].iloc[-i]
  f,g,h,I=temp2[['payoff_c', 'prob', 'Expected','Actual']].iloc[-i]
  df3[str(i)] = 0
  df3.loc[[20,21,22,23],str(i)] = [int(f),int(g*100),int(h*100-100),I]
  df4.loc[i-1,6] = int(f)
  counter =0
  for j in [a,b,c,d,e]:
    if j!=4:
      df3.loc[counter*4+j,str(i)] =1
      name2 = df3.loc[counter*4+j,'name']
      df4.loc[i-1,counter+1] = name2
    counter += 1


df2=df[['Arena','name','O_odds',
        'C_odds','pred',
        'Expected_profit','win']]
df2['O_odds'] = df2['O_odds'] .astype(np.int32)
df2['C_odds'] = df2['C_odds'] .astype(np.int32)
df2['pred'] = np.round(df2['pred'],3)
df2['Expected_profit']=np.round(df2['Expected_profit'],3)

#print(df2)
#print(df3[['Arena','name']+list(map(str,range(1,len(temp2)+1)))])
temp2.index= range(1,len(temp2)+1)[::-1]
temp2 = temp2[temp2.index <= 10]
for i in range(1,1+len(temp2)):
    a,b,c,d,e=temp2[range(5)].iloc[-i]
    counter =0
    for j in [a,b,c,d,e]:
        if j!=4:
            None#print(df.loc[counter*4+j][['name','close_odds','Expected_profit_at_close']].values)
        else: None#print(None)
        counter += 1
    odds = temp2[['payoff_c','Expected','Actual']].iloc[-i]
    #print(odds,'\n')
#print ('Conclusion')
try:
  if 0 in  data['winners']:
    winnings = np.nan
  else:
    winnings= 0
except : winnings=np.nan
for i in range(1,1+len(temp2)):
    a,b,c,d,e=temp2[range(5)].iloc[-i]
    counter =0
    if temp2['Actual'].iloc[-i] == True:
      odds = temp2['payoff_c'].iloc[-i]
      winnings += odds
#print(winnings)
'''
automate makingof bets
'''
dictt = {'Scurvy Dan the Blade': 1,
         'Young Sproggie': 2,
         'Orvinn the First Mate': 3,
         'Lucky McKyriggan': 4,
         'Sir Edmund Ogletree': 5,
         'Peg Leg Percival': 6,
         'Bonnie Pip Culliford': 7,
         'Puffo the Waister': 8,
         'Stuff-A-Roo': 9,
         'Squire Venable': 10,
         'Captain Crossblades': 11,
         "Ol' Stripey": 12,
         'Ned the Skipper': 13,
         'Fairfax the Deckhand': 14,
         'Gooblah the Grarrl': 15,
         'Franchisco Corvallio': 16,
         'Federismo Corvallio': 17,
         'Admiral Blackbeard': 18,
         'Buck Cutlass': 19,
         'The Tailhook Kid': 20}
temp3 = temp2.copy()
for i in range(0,20,4):
  pirates = df.loc[i:i+3].reset_index(drop=1)
  pirates_dictt = pirates['name'].map(dictt).to_dict()
  pirates_dictt[4] = ''
  temp3[i//4] = temp3[i//4].map(pirates_dictt)

paste = []
if True:
  f = open('1b_bet.sh','w')
  for i in range(min(len(temp3),10),0,-1):
    bet_details = temp3.loc[i]
    days_played = date.today()-date(2003,1,8)
    #bet_amt = (name-1345)*2+50
    total_odds = bet_details['payoff_c']
    winnings = total_odds*bet_amt
    curlA = 'http://www.neopets.com/pirates/process_foodclub.phtml?'
    input_list = ['', '']
    for arena in range(5):
      pirate = bet_details[arena]
      if pirate != '':
           input_list[0] = input_list[0] + "winner%s=%s&"%(str(arena+1),pirate)
    for arena in range(5):
      pirate = bet_details[arena]
      if pirate != '':
           input_list[1] = input_list[1] + "matches[]=%s&"%(str(arena+1))
    input_list[0] = input_list[0][:-1]
    input_list[1] = input_list[1][:-1]
    actual_amt = int(min(bet_amt,1000000/total_odds))
    input_list = input_list+[actual_amt,
                             str(total_odds),
                             min(1000000,winnings)]
    curlB = "%s&%s&bet_amount=%s&total_odds=%s&winnings=%s&type=bet'  \n"%tuple(input_list)
    f.write(curlA + curlB )
    f.write('\n\nsleep 5\n\n')
    paste += [curlA+curlB + '\n',]
print('finished calculating round',Round)




finished calculating round 7722


In [8]:
'''
Here writes each pirates' opening, close odds, predicted winning rate and expected profit, and if that pirate has won
Expected profit = Close_odds * predictet_win_rate , the higher the better
'''
print (df2)

     Arena                   name  O_odds  C_odds   pred  Expected_profit  win
0    ShpWr           Buck Cutlass       4       4  0.210           -0.161  NaN
1    ShpWr     Gooblah the Grarrl       4       4  0.225           -0.101  NaN
2    ShpWr       The Tailhook Kid       3       3  0.285           -0.145  NaN
3    ShpWr   Scurvy Dan the Blade       3       3  0.281           -0.158  NaN
4      Lgn       Peg Leg Percival      13      13  0.074           -0.034  NaN
5      Lgn   Franchisco Corvallio       2       2  0.437           -0.126  NaN
6      Lgn         Squire Venable      13      13  0.046           -0.402  NaN
7      Lgn    Sir Edmund Ogletree       2       2  0.443           -0.115  NaN
8     TrIs   Fairfax the Deckhand       7       8  0.121           -0.028  NaN
9     TrIs      Puffo the Waister       5       6  0.162           -0.026  NaN
10    TrIs    Captain Crossblades       7       8  0.112           -0.100  NaN
11    TrIs     Admiral Blackbeard       2       2  0

In [4]:
'''
The best arenas calculated by expected profit
'''
print(df3[['Arena','name']+list(map(str,range(1,len(temp2)+1)))])

     Arena                   name    1    2    3    4    5    6    7    8  \
0    ShpWr       Peg Leg Percival    0    0    0    0    0    0    0    0   
1    ShpWr   Franchisco Corvallio    0    0    0    0    0    0    1    0   
2    ShpWr   Fairfax the Deckhand    0    0    0    0    0    0    0    0   
3    ShpWr       Lucky McKyriggan    1    0    1    0    1    0    0    0   
4      Lgn   Bonnie Pip Culliford    0    0    0    0    0    0    0    0   
5      Lgn      Puffo the Waister    0    0    0    0    0    0    0    0   
6      Lgn    Sir Edmund Ogletree    0    0    0    0    0    0    0    1   
7      Lgn       The Tailhook Kid    1    1    1    1    1    1    1    0   
8     TrIs         Young Sproggie    0    0    0    0    0    0    0    0   
9     TrIs         Squire Venable    0    0    0    0    0    0    0    0   
10    TrIs    Federismo Corvallio    1    1    1    1    1    1    1    1   
11    TrIs            Stuff-A-Roo    0    0    0    0    0    0    0    0   

In [5]:
# Do not click unless you want to bet on pirate(s). If you bet in an invalid round, you will waste your NP
for i in paste:
    print (i[:-1])

http://www.neopets.com/pirates/process_foodclub.phtml?winner1=4&winner2=5&winner3=17&winner4=15&matches[]=1&matches[]=2&matches[]=3&matches[]=4&bet_amount=12772&total_odds=48&winnings=613056&type=bet'  

http://www.neopets.com/pirates/process_foodclub.phtml?winner2=7&winner3=17&winner4=15&matches[]=2&matches[]=3&matches[]=4&bet_amount=12772&total_odds=52&winnings=664144&type=bet'  

http://www.neopets.com/pirates/process_foodclub.phtml?winner2=5&winner3=17&winner4=15&winner5=1&matches[]=2&matches[]=3&matches[]=4&matches[]=5&bet_amount=12772&total_odds=48&winnings=613056&type=bet'  

http://www.neopets.com/pirates/process_foodclub.phtml?winner1=16&winner2=20&winner3=17&winner4=15&winner5=1&matches[]=1&matches[]=2&matches[]=3&matches[]=4&matches[]=5&bet_amount=12772&total_odds=48&winnings=613056&type=bet'  

http://www.neopets.com/pirates/process_foodclub.phtml?winner2=20&winner3=17&winner4=15&winner5=19&matches[]=2&matches[]=3&matches[]=4&matches[]=5&bet_amount=12772&total_odds=24&winni